<b>Audio and Music Processing Lab - Module 2</b><br>Gonzalo Nieto<br>Morgan Buisson<br>9.03.2021
## AMPLab2 - Ethnomusicology research
### Melodic analysis
The aim of this notebook is to count interval types per poem in all recordings available. It uses pre-processed data with the ```Pre-processing.ipynb``` notebook. Note that there might be paths that need to be changed. 

In [5]:
from music21 import *
import json
import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Create a dictionary with poem identifier and notes included. The measure number is the fourth element in the list of each poem (index 3), the poem indentifier is the first one (index 0).

In [6]:
def recording_notes_dict(rec_filename, mbid):
    # load recording data and metadata (starting measure of each poem)
    filename = rec_filename
    with open(filename, "rb") as f:
        recording = pickle.load(f)

    # load score
    s_path = "./arab-andalusian-music/scores-musicxml/"+mbid+".xml"
    s = converter.parse(s_path)

    # save each poem notes with their name 
    print(f'\nCreating dictionary of recording {mbid}')
    poemsNotes = {}
    for i in range(len(recording)):
        if recording[i][1] != 'Not Retrieved/Instrumental' and nawba[i][-1] =='qaṣīdah': 
            poemName = recording[i][0]
            startMeasure = recording[i][3]
            if i== len(recording)-1:
                endMeasure = s.parts[0].measure(-1).number
            else:
                endMeasure = recording[i+1][3]
            
            poemMeasures = s.measures(startMeasure,endMeasure)
            poemNotes = poemMeasures.flat.notesAndRests.stream() 
            poemsNotes[poemName] = poemNotes
            print(f'- {poemName}: measures {startMeasure} to {endMeasure} (number of notes = {len(poemNotes)})')
    return poemsNotes


Count interval types for each poem.

In [15]:
def melody_intervals(poemsNotes):
    print('\nCounting intervals...')
    # Init empty dictionary
    intervals = {}
    for poemName in poemsNotes:
        intervals[poemName] = {}

    # Fill dictionary with interval counts
    for poemName in poemsNotes:
        for n in poemsNotes[poemName][:-1]:
            # All notes are included, also grace notes
            if n.isNote and n.next().isNote: # only count if the notes are consecutive
                itv = interval.Interval(n, n.next())
                intervals[poemName][itv.name] = intervals[poemName].get(itv.name, 0) + 1

    print('Done counting intervals!')
    print(intervals)
    return intervals

Plot an interval histogram for each of the sections to compare them.

In [16]:
def plot_intervals_histogram(intervals):
    for poemName in intervals:
        sectionDic = intervals[poemName]
        # Order the intervals
        # Create a dictionary with the equivalence of each interval's size in semitones and its name.
        intervalsOrder = {}
        for k in sectionDic.keys():
            itv = interval.Interval(k)
            intervalsOrder[itv.semitones] = k
        # Order list of intervals by semitones size
        xValues = sorted(intervalsOrder.keys())
        # Order list of interval names by their semitiones size to be use as ticks for the x axis.
        xTicks = [intervalsOrder[i] for i in xValues]
        # Order list of y axis values
        yValues = np.array([sectionDic[i] for i in xTicks])
        # Normalize yValues for better comparison
        yValues = yValues / sum(yValues)
        
        # Create the plot
        plt.figure()
        plt.bar(xValues, yValues)
        plt.xticks(xValues, xTicks)
        # Common x and y axes limits
        plt.xlim(-1, 13)
        plt.ylim(0, 1)
        plt.title(poemName)

    plt.tight_layout()
    plt.show()

In [17]:
def save_dict(dict, outputFolder, mbid):
    file = json.dumps(dict)
    f = open(f"{outputFolder}/{mbid}_intervals.json","w")
    f.write(file)
    f.close()

Count intervals for all recordings:

In [49]:
outputFolder = './intervals'

for root, dirs, files in os.walk("./preprocessed_recordings", topdown=False):
   for name in files:
       if name != '.DS_Store':
            filename = os.path.join(root, name)
            mbid = name[:-4]
            poemsNotes = recording_notes_dict(filename, mbid)
            intervals = melody_intervals(poemsNotes)
            save_dict(intervals, outputFolder, mbid)

52, 'M3': 15}}

Creating dictionary of nawba 62acb9e5-e311-40ab-9c5c-866d071b5c93
- Bidimam al-hawa: measures 149 to 292 (number of notes = 1793)

Counting intervals...
Done counting intervals!
{'Bidimam al-hawa': {'P1': 177, 'M2': 675, 'm2': 424, 'M3': 41, 'P4': 39, 'm3': 18, 'P5': 6}}

Creating dictionary of nawba 6311c0de-d66f-482d-8dd1-8d8917ca6a76
- Tahya bikum: measures 168 to 216 (number of notes = 305)
- Wa nurukum: measures 216 to 358 (number of notes = 825)
- Kullu saabin: measures 366 to 532 (number of notes = 982)
- Malikun tafarrad: measures 777 to 961 (number of notes = 708)
- Ayyu dabyin ala: measures 970 to 1055 (number of notes = 298)
- Allahu yalamu anna: measures 1055 to 1124 (number of notes = 235)

Counting intervals...
Done counting intervals!
{'Tahya bikum': {'P1': 53, 'M3': 12, 'M2': 137, 'm3': 9, 'm2': 39, 'P4': 3}, 'Wa nurukum': {'P1': 88, 'M2': 356, 'M3': 28, 'm2': 129, 'P4': 20, 'm3': 26}, 'Kullu saabin': {'P1': 138, 'M2': 408, 'm2': 131, 'm3': 48, 'P4': 9, 